In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import json
import os
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from sys import getsizeof
from tqdm import tqdm

### 1- Testing creating 240x240x160x4 Brain Tumor task 01 files from 240x240x155x4 files (with zero padding)

In [2]:
mri = nib.load("../../datasets/Task01_BrainTumour/train/images/BRATS_001.nii.gz").get_fdata().astype(np.float32)

In [3]:
mri.shape

(240, 240, 155, 4)

In [4]:
getsizeof(mri) # 142.848144 MBs

142848144

In [5]:
new_mri = np.zeros((240, 240, 160, 4)).astype(np.float32)

In [6]:
new_mri.shape

(240, 240, 160, 4)

In [7]:
new_mri[:, :, :mri.shape[2], :] = mri[:, :, :, :]

In [8]:
new_mri.shape

(240, 240, 160, 4)

In [9]:
getsizeof(new_mri) # 147.456144 MBs

147456144

#### Visualizing Original MRI file and new padded MRI file

In [10]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=new_mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    if layer > mri.shape[2] - 1:
        plt.imshow(mri[:, :, mri.shape[2] - 1, channel], cmap='gray');
    else:
        plt.imshow(mri[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    plt.imshow(new_mri[:, :, layer, channel], cmap='gray');
    plt.title("New MRI", fontsize=20)
    plt.axis('off')

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 2- Testing splitting a 320x320x130 MRI file into 96 layer chunks with the last chunk padded with zeros to layer size 96

In [11]:
mri = nib.load("../../datasets/Task02_Heart/train/images/la_003.nii.gz").get_fdata().astype(np.float32)

In [12]:
mri.shape

(320, 320, 130)

In [13]:
mri.shape[2] / 96

1.3541666666666667

In [14]:
mri.shape[2] % 96 # Modulus does not equal zero => leftover chunk to be padded with zeros

34

In [15]:
num_chunks = int(np.ceil(mri.shape[2] / 96)) # Two chunks

In [16]:
num_chunks

2

In [17]:
layer_size = 96

chunks = np.zeros((num_chunks, mri.shape[0], mri.shape[1], layer_size))

for i in range(num_chunks):
    if i == num_chunks - 1: # last iteration (if we wish to pad the last chunk with zeros and use it)
        chunks[i, :, :, :mri.shape[2] % layer_size] = mri[:, :, i*layer_size: i*layer_size + (mri.shape[2] % layer_size)]
    else:
        chunks[i, :, :, :] = mri[:, :, i*layer_size: (i+1)*layer_size]    
    
    nib.save(nib.Nifti1Image(chunks[i, :, :, :], np.eye(4)), f"la_003_{i}.nii.gz") # Numpy array has to be set as Nifti Image object before saving

In [18]:
chunks.shape

(2, 320, 320, 96)

In [19]:
# Create layer slider
select_layer = IntSlider(min=0, max=mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    plt.imshow(mri[:, :, layer], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=129), Output…

In [20]:
# Create layer slider
select_layer = IntSlider(min=0, max=chunks[0, :, :, :].shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    plt.imshow(chunks[0, :, :, layer], cmap='gray');
    plt.title("Chunk 1 MRI", fontsize=20)
    plt.axis('off')
    
# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=95), Output(…

In [21]:
# Create layer slider
select_layer = IntSlider(min=0, max=chunks[1, :, :, :].shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    plt.imshow(chunks[1, :, :, layer], cmap='gray');
    plt.title("Chunk 1 MRI", fontsize=20)
    plt.axis('off')
    
# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=95), Output(…

#### Testing if shapes of saved files are correct

In [22]:
mri_1 = nib.load("la_003_0.nii.gz").get_fdata()

In [23]:
mri_1.shape

(320, 320, 96)

In [24]:
mri_2 = nib.load("la_003_1.nii.gz").get_fdata()

In [25]:
mri_2.shape

(320, 320, 96)